# Monoids for Pythonistas

[Bartosz Milewsky](https://bartoszmilewski.com/) has a great series of articles on category theory as it applies to programmers.
I'm not going to lie, the material is pretty dense which is par for the course for a lot of mathematics subjects (maybe it's worth mentioning that Bartosz has a PhD in Theoretical Physics 😬).
However, many of the core concepts are reasonably simple abstractions, and programmers know all about abstractions!
The language can be as big a hurdle as reading mathematics symbols, but these are the tools mathematicians use to convey ideas in an elegant and concise way.

I promise that you know what monoids are - you use them every day as a programmer!
I am going to flip the script and try to work my way to the definition instead of the other way around.
Hopefully, with a bit of patience, we can uncover the power of these ideas and see how they apply to our day-to-day jobs!

**Disclaimer** This is not meant to be a mathematically rigorous discussion.  Many small details are overlooked in favor of a high-level understanding.

## Sets

As a programmer, you _should_ know what a set is - an unordered collection of unique items of the same type (and if not, maybe a blog post for another day!).

```python
>>> my_pets = {"Lamby", "Dozer", "Onyx", "BlackHen", "RedHen", "NoisyRooster"}
>>> my_pets.add("BuffHen")
>>> "Frank" in my_pets
False
>>> my_pets.add("BuffHen")  # `add` is idempotent and won't create a duplicate
>>> my_pets  # Notice that the order isn't consistent
{'NoisyRooster', 'RedHen', 'BuffHen', 'BlackHen', 'Lamby', 'Onyx', 'Dozer'}
>>>
>>> pets_son_feeds = {"Shayna", "Lamby"}
>>> pets_son_feeds.intersection(my_pets)
{'Lamby'}
>>>
```

The integers form a set, $\mathbb{I} = \{-1,0,1,2,3,4...\}$.
Rational numbers form a set, $\mathbb{Q} = \{-99.42,0,1.1,1.2,1.3...\}$.
Character strings even form a set, $\{\text{"a", "b", "c", "foo", "cow", "Bob", ...}\}$.
For the purposes of this article, a "set" is a mathematical set, not constrained by physical memory limits of computers.

## Symbols

Symbols are the syntax of mathematics.

### $\forall$

Means "for all".  Given a set, "_for every element_ in the set."  In the definition of a monoid, we will use this to assert a property of the items in the set.  It is somewhat analogous to a `for` loop over all the items.

```python
>>> odds = {1, 3, 5, 7, 9}
>>> for odd in odds:  # ∀ in the set...
...   assert odd % 2 == 1  # Assert that an odd divided by 2 has a remainder of 1
...
>>> # Success!
```

### $\in$

means "in" or "an element of".  1 is _an element of_ the set of integers.

```python
>>> some_primes = {2, 3, 5, 7, 11}
>>> 7 in some_primes  # 7 ∈ some_primes == True
True
>>>
```

## More terminology

### Binary operation

An operation that works on two (hence binary) values; addition, multiplication, division, etc.  Most of the Python numeric [dunder methods](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types) are binary operations.

### Associative

If an operation is associative, the order of _composition_ doesn't matter.
This property is **very** important as we'll see a bit later.
- Addition **is** associative.  1 + (2 + 3) is the same as (1 + 2) + 3.
- String concatenation **is** associative.  "A" + ("B" + "C") == ("A" + "B") + "C"
  - Unlike numeric addition where 1 + 2 == 2 + 1, string concatenation is **not** _commutative_!  Monoids only depend on the associative property.  (See [Andrey Mokhov's article](https://blogs.ncl.ac.uk/andreymokhov/united-monoids/) for a fascinating deep-dive into this subject)
- Subtraction is **not** associative as order of composition matters.  1 - (2 - 3) is **not equal** to (1 - 2) - 3.


### Closed

For a given operation, the input and output types must match for the operation to be closed.
- The minimum of two integers maps back into the set of integers.

```python
def min_(a: int, b: int) -> int:
    return min(a, b)
```

- Multiplication of two integers stays within the set of integers.

```python
>>> type(42 * 7)
<class 'int'>
>>> a = 42
>>> b = 7
>>> type(a)
<class 'int'>
>>> type(b)
<class 'int'>
>>> type(a * b)
<class 'int'>
>>>
```

- Floating point division of two integers returns a float, which is outside the set of integers.  The _input_ and _output_ types don't match.  This is **not** a closed operation.

```python
>>> type(42 / 33)
<class 'float'>
>>>
```

- The `chr` function in Python takes an integer and returns a `str`.  This operation is **not** closed for the same reason; the output is of a different type than the input.

```python
>>> type(chr(51))
<class 'str'>
>>>
```


## Axioms

We're almost there!  We need to mention identity.  The importance of identity is a bit of a rabbit hole, so I'll leave it as an exercise for the reader to dig deeper (abstract algebra rings and rngs).

An identity element is an element that does not change the output when paired with any other input.

### Identity

- Numeric addition: 0 is the identity (works with reals, integers, naturals, etc).

```python
>>> 42 + 0
42
>>>
```

- Numeric multiplication: 1 is the identity (gotcha!  The identity is dependent on the _operation_ and the _type_).

```python
>>> 42 * 1
42
>>>
```

- List concatenation: the empty list is the identity

```python
>>> [1, 4, 3] + []
[1, 4, 3]
>>>
```

The reason we need this is a little subtle.
We are using a binary operation to make a computation, but what do we do when we only have one value to work with?
Many of the use cases of monoids are recursive in nature, so what do you do when you reduce a list to one element?
This is fairly representative of the problem (or imagine if `functools.reduce` didn't take a default argument).

```python
>>> list(map(lambda x, y: x + y, [42]))
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: <lambda>() missing 1 required positional argument: 'y'
>>>
```

We can use the identity!
This allows the developer to safely perform a binary operation on a single element without changing the output.


## Definition

Hopefully, we now have the tools to unpack this definition...

According to Eric Weisstein on Wolfram, a monoid is "a set that is closed under an associative binary operation and has an identity element $I \in S$ such that $ \forall a \in S, Ia=aI=a$".

I don't like that definition.
What the heck does $Ia=aI=a$ mean?
It _looks_ like multiplication, but that is one of many possible binary operations.
Here's my rephrasing:  
A monoid consists of a set, $S$, that is closed under an associative binary operation, $f$, and has an identity element $I \in S$ such that $ \forall a \in S, f(I, a)=f(a, I)=a$.

A monoid has three components.
- The set $S$, often called the "carrier set".  Programmers know this as a type (int, float, double, etc).
- $I$ is the identity element, which we identified in the previous section
- A binary operation.  Addition, multiplication, list concatenation, etc.

I said in the intro that you already know what a monoid is.
Here are some examples:
- Integers, addition, 0
- Reals, multiplication, 1
- Lists, concatenation, \[ \]

## Why it matters

At this point, you might be thinking about whipping me with a large stick for describing simple arithmetic in such obtuse and abstract terms.
It matters, I promise!
These definitions allow us to precisely identify objects with certain properties that support certain operations.
Enough math, why does this matter to programmers?!

### Divide and conquer

Remember how I said that associative property would be important later on?
Consider the following items... they're all equivalent.

    1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10
    (1 + 2) + (3 + 4 + 5 + 6 + 7 + 8 + 9 + 10)
    (1 + (2 + (3 + (4 + (5 + (6 + (7 + (8 + (9 + 10)))))))))
    (((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

Here's another example.

    "a" + "b" + "c" + "d" + "e" + "f" + "g" + "h" + "i" + "j"
    "ab" + "cd" + "ef" + "gh" + "ij"
    "abcd" + "efgh" + "ij"
    "abcdefgh" + "ij"
    "abcdefghij"

I can write those down on a set of index cards and pass the pairs to two people at a time (think two CPU cores).
As long as I track the order of the results, I can recursively continue this process in any order of composition until the task is done.
This process succeeds even if the workers have different computational capacities.
The answer is the same as if one poor individual toiled away at the tasks independently.

### More cores, more computers

If the list is really long or the binary operation is non-trivial, we can **safely** and easily divide this tasks amongst different cores - or even different computers.
Notice that we are talking about _CPU_-bound tasks, not _IO_-bound tasks (multiple cores do little to alleviate the latter).
Python doesn't have the ability to do this natively, but other languages do.

- Python3, single core

```shell
± \time python -c "print(sum(range(1000000001)))"
500000000500000000
      107.69 real        37.33 user        59.45 sys
```

- Python3, 16 cores

```python
from multiprocessing import Pool
from functools import partial

MAX_INT = 1000000001
N_PROC = 16

def sum_range(n: int, offset: int) -> int:
    high = n + offset if n + offset < MAX_INT else MAX_INT
    return sum(range(n, high))

if __name__ == '__main__':
    with Pool(processes=N_PROC) as pool:
        offset = MAX_INT // N_PROC
        print(sum(pool.map(partial(sum_range, offset=offset), range(0, MAX_INT, offset))))

    print("Complete")
```

```shell
/usr/bin/time python3 pool.py
500000000500000000
Complete
        3.21 real        47.68 user         1.03 sys
```

This divide-and-conquer concept is critical to well-known large-scale algorithms like [MapReduce](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html).

It doesn't mean we can't do it ourselves, though!
Libraries like `multiprocessing` allow developers to delegate tasks among different cores or processors.
Keep in mind that just like delegating tasks to friends, there is a computational overhead to splitting up tasks.
It is important to understand that libraries like [`Numpy`](https://numpy.org/) don't [get you around the single-core limit](https://scipy.github.io/old-wiki/pages/ParallelProgramming).
Do your research!
Know your data and the problem domain before you start concatenating small strings across processes! 😜

## References

- Weisstein, Eric W. "Monoid." From MathWorld--A Wolfram Web Resource. https://mathworld.wolfram.com/Monoid.html 
- https://en.wikipedia.org/wiki/Monoid
- https://wiki.haskell.org/Monoid
- https://github.com/dry-python/functional-jargon-python
- https://planetmath.org/examplesofnoncommutativeoperations
- https://blogs.ncl.ac.uk/andreymokhov/united-monoids/

In [31]:
from statistics import median
from toolz.itertoolz import take

nums = [42, 3, 1, 5, 66, 7, 8, 9, 22, 35, 99]

last = nums[0]
for idx, num in enumerate(nums, 1):
    if idx % 2:
        med = last
    else:
        med = (num + last) / 2
        last = num
    print(med, sorted(nums[:idx]), median(nums[:idx]))

median(nums)

42 [42] 42
22.5 [3, 42] 22.5
3 [1, 3, 42] 3
4.0 [1, 3, 5, 42] 4.0
5 [1, 3, 5, 42, 66] 5
6.0 [1, 3, 5, 7, 42, 66] 6.0
7 [1, 3, 5, 7, 8, 42, 66] 7
8.0 [1, 3, 5, 7, 8, 9, 42, 66] 7.5
9 [1, 3, 5, 7, 8, 9, 22, 42, 66] 8
22.0 [1, 3, 5, 7, 8, 9, 22, 35, 42, 66] 8.5
35 [1, 3, 5, 7, 8, 9, 22, 35, 42, 66, 99] 9


9